In [1]:
%load_ext boa.ipython
import boa
boa.interpret.set_cache_dir()
boa.env.enable_gas_profiling()

In [82]:
%%vyper

@external
@pure
def cbrt(x: uint256) -> uint256:

    if x == 0:
        return 0

    # multiply with 10 ** 36 for increasing cbrt precision
    _x: uint256 = unsafe_mul(x, 10**36)

    # get log2(x) for approximating initial value
    # logic is: cbrt(a) = cbrt(2**(log2(a))) = 2**(log2(a) / 3) ≈ 2**|log2(a)/3|
    # from: https://github.com/transmissions11/solmate/blob/b9d69da49bbbfd090f1a73a4dba28aa2d5ee199f/src/utils/FixedPointMathLib.sol#L352

    a_pow: int256 = 0
    if _x > 340282366920938463463374607431768211455:
        a_pow = 128
    if unsafe_div(_x, shift(2, a_pow)) > 18446744073709551615:
        a_pow = a_pow | 64
    if unsafe_div(_x, shift(2, a_pow)) > 4294967295:
        a_pow = a_pow | 32
    if unsafe_div(_x, shift(2, a_pow)) > 65535:
        a_pow = a_pow | 16
    if unsafe_div(_x, shift(2, a_pow)) > 255:
        a_pow = a_pow | 8
    if unsafe_div(_x, shift(2, a_pow)) > 15:
        a_pow = a_pow | 4
    if unsafe_div(_x, shift(2, a_pow)) > 3:
        a_pow = a_pow | 2
    if unsafe_div(_x, shift(2, a_pow)) > 1:
        a_pow = a_pow | 1

    # initial value:
    a: uint256 = unsafe_div(unsafe_mul(pow_mod256(2, unsafe_div(convert(a_pow, uint256), 3)), 1260), 1000)

    # 6 newton-raphson iterations:
    a = unsafe_div(unsafe_add(unsafe_mul(2, a),unsafe_div(_x, a**2)), 3)
    a = unsafe_div(unsafe_add(unsafe_mul(2, a),unsafe_div(_x, a**2)), 3)
    a = unsafe_div(unsafe_add(unsafe_mul(2, a),unsafe_div(_x, a**2)), 3)
    a = unsafe_div(unsafe_add(unsafe_mul(2, a),unsafe_div(_x, a**2)), 3)
    a = unsafe_div(unsafe_add(unsafe_mul(2, a),unsafe_div(_x, a**2)), 3)
    a = unsafe_div(unsafe_add(unsafe_mul(2, a),unsafe_div(_x, a**2)), 3)

    return a

In [20]:
%%vyper

@external
@pure
def _sum(arr: uint256[3]) -> uint256:
    s: uint256 = 0
    # for i in range(3):
    #     s += arr[i]
    s = arr[0] + arr[1] + arr[2]
    return s

In [21]:
c = _.deploy()

In [22]:
val = 10**41

# print(
#     f'Vyper:  {c.cbrt(val)}\n'
#     f'Python: {_cbrt_wad_ideal(val)}\n'
#     f'Gas:    {c._computation.get_gas_used()}\n'
# )

c._sum([10**18, 10**18, 10**18])

# c.line_profile().summary()
c._computation.get_gas_used()

222